In [1]:
import pandas as pd
import numpy as np
import time
import os
from openpyxl import load_workbook
from glob import glob
from ModeloMatematico1_MultiplosEc import ModeloMatematico as ModelFleetSize_MultiplosEc
from ModeloMatematico2_MultiplosEc import ModeloMatematico as ModelConsumo_MultiplosEc
from ModeloMatematico1_Fisico import ModeloMatematico as ModelFleetSize_Fisico
from ModeloMatematico2_Fisico import ModeloMatematico as ModelConsumo_Fisico

In [2]:
# Caminho do arquivo Excel
caminho_arquivo = f"./Resultados_Hibrido.xlsx"

# Ler todas as abas
dados = []
instancias = []
resumo = []
# Mostrar os nomes das abas e seus respectivos DataFrames
atualizar = True
if atualizar:
    abas = pd.read_excel(caminho_arquivo, sheet_name=None)
    for nome_aba, df in abas.items():
        #print(f'Aba: {nome_aba}')
        #print(df.head())  # Exibe as primeiras linhas do DataFrame
        dado = df.to_dict(orient="records")
        #print(dado)
        if nome_aba != "resumo":
            dados.append(dado)
            instancias.append(nome_aba)
        else:
            resumo = dado

In [3]:
# Funcao
def Get_Di(Pontos, file):
    df = pd.read_excel("DELIVERIES.xlsx")
    df['data'] = pd.to_datetime(df['tour_date'])
    df['dia'] = df['data'].dt.day
    df['mes'] = df['data'].dt.month
    df['ano'] = df['data'].dt.year

    data = str(file).replace(".xlsx", "")[-10:]
    dia = int(data[-2:])
    mes = int(data[-5:-3])
    ano = int(data[:4])
    
    df_demandas = df[df['ano']==ano]
    df_demandas = df_demandas[df_demandas['mes']==mes]
    df_demandas = df_demandas[df_demandas['dia']==dia]
    dict_demanda = df_demandas.to_dict(orient="records")
    
    d_i = np.zeros(len(Pontos))

    for d in dict_demanda:
        client_id = d["client_id"]
        weight_kg = d["weight_kg"]

        if client_id not in Pontos: continue
        i = Pontos.index(client_id)
        d_i[i] = weight_kg
        
    

    return d_i


In [4]:
def combinacoes_2_a_2(conjunto, k=2):
    # Converte o conjunto para uma lista para facilitar o acesso por índice
    lista = list(conjunto)
    resultado = []
    
    def gerar_combinacoes(prefixo, inicio, k=2):
        # Quando o prefixo tem 2 elementos, adiciona ao resultado
        if len(prefixo) == k:
            resultado.append(tuple(prefixo))
            return
        
        # Loop para escolher os elementos
        for i in range(inicio, len(lista)):
            # Adiciona o próximo elemento ao prefixo e chama recursivamente
            gerar_combinacoes(prefixo + [lista[i]], i + 1, k)

    # Chama a função recursiva inicial
    gerar_combinacoes([], 0, k)
    return resultado

def CriarConjuntoCorteHiperplane(qntClientes):
    S = []
    __S = lambda __n: [list(s) for s in combinacoes_2_a_2(list(range(1, qntClientes+1)), __n)]
    for __n in range(2, 4):
        list_S = __S(__n)
        S += list_S
        
    return S


In [5]:
# Leitura de Arquivos
CONJUNTO_CORTES = dict()

INSTANCIAS = []

for file in glob("Instancias Clusters/*.xlsx"):
    try:
        df = pd.read_excel(file, "Pontos")

        def converter_id(valor):
            if valor == "CD":
                return valor  # Retorna "CD" sem alterar
            try:
                return float(valor)  # Tenta converter o valor para número
            except ValueError:
                return valor  # Se não for possível converter, retorna o valor original

        # Aplicar a função à coluna 'ID'
        df['ID'] = df['ID'].apply(converter_id)

        df_Energia = pd.read_excel(file, "Tabela Energia", usecols=['origin', 'destination', 'distancia (m)', 'e_ij_fixo', 'e_ij'])
        dict_Energia = df_Energia.to_dict(orient='records')    
        clusters = df["Cluster"].unique()

        CD = df[df["ID"]=="CD"]
        CD = list(CD["ID"].to_list())[0]
        for cluster in clusters:
            if cluster == "CD": continue
            clientes = df[df["Cluster"]==cluster]
            clientes = list(clientes["ID"].to_list())

            if len(clientes) not in CONJUNTO_CORTES:
                print("CRIANDO CORTES", len(clientes))
                print(file)
                CONJUNTO_CORTES[len(clientes)] = CriarConjuntoCorteHiperplane(len(clientes))
            
            # PONTOS
            DADO = {"file": file, "cluster": cluster, "PONTOS": [CD] + clientes}
            

            # DADOS DISTANCIA
            d_ij = np.zeros(shape=(len(DADO["PONTOS"]), len(DADO["PONTOS"])))
            e_FIXO_ij = np.zeros(shape=(len(DADO["PONTOS"]), len(DADO["PONTOS"])))
            e_ij = np.zeros(shape=(len(DADO["PONTOS"]), len(DADO["PONTOS"])))

            for d in dict_Energia:
                i = d["origin"]
                j = d["destination"]
                
                if i not in DADO["PONTOS"]: continue
                if j not in DADO["PONTOS"]: continue
                
                idx_i = DADO["PONTOS"].index(i)
                idx_j = DADO["PONTOS"].index(j)

                d_ij[idx_i, idx_j] = d["distancia (m)"]
                e_FIXO_ij[idx_i, idx_j] = d["e_ij_fixo"] 
                e_ij[idx_i, idx_j] = d["e_ij"]

            d_ij[d_ij == 0] = np.inf
            e_FIXO_ij[e_FIXO_ij == 0] = np.inf
            e_ij[e_ij == 0] = np.inf
            
            # ATENCAO TAXA DE AUMENTO DE DISTANCIA ABAIXO:
            d_ij = d_ij / 1000 * 1.2215
            DADO["d_ij"] = d_ij
            DADO["e_fixo_ij"] = e_FIXO_ij
            DADO["e_ij"] = e_ij
            
            DADO["d_i"] = Get_Di(DADO["PONTOS"], file)
            INSTANCIAS.append(DADO.copy())
    except:
        print("FALHA EM UM ARQUIVO!!!")
        print(file)
    try:
        del df, df_Energia, dict_Energia, clusters, CD, clientes, d_ij, e_FIXO_ij, e_ij, DADO
    except:
        pass

CRIANDO CORTES 58
Instancias Clusters\tabelas_2023-09-06.xlsx
CRIANDO CORTES 22
Instancias Clusters\tabelas_2023-09-06.xlsx
CRIANDO CORTES 48
Instancias Clusters\tabelas_2023-09-06.xlsx
CRIANDO CORTES 37
Instancias Clusters\tabelas_2023-09-06.xlsx
CRIANDO CORTES 49
Instancias Clusters\tabelas_2023-09-06.xlsx
CRIANDO CORTES 31
Instancias Clusters\tabelas_2023-09-15.xlsx
CRIANDO CORTES 40
Instancias Clusters\tabelas_2023-09-15.xlsx
CRIANDO CORTES 64
Instancias Clusters\tabelas_2023-09-15.xlsx
CRIANDO CORTES 35
Instancias Clusters\tabelas_2023-09-15.xlsx
CRIANDO CORTES 34
Instancias Clusters\tabelas_2023-09-20.xlsx
CRIANDO CORTES 84
Instancias Clusters\tabelas_2023-09-20.xlsx
CRIANDO CORTES 60
Instancias Clusters\tabelas_2023-09-20.xlsx
CRIANDO CORTES 36
Instancias Clusters\tabelas_2023-09-20.xlsx
CRIANDO CORTES 13
Instancias Clusters\tabelas_2023-09-20.xlsx
CRIANDO CORTES 53
Instancias Clusters\tabelas_2023-09-22.xlsx
CRIANDO CORTES 50
Instancias Clusters\tabelas_2023-09-22.xlsx
CRIANDO 

In [6]:
def vrp_vizinho_mais_proximo(pontos, num_veiculos, distancia, d_i):
    rotas = [[] for _ in range(num_veiculos)]  # Listas de rotas para cada veículo
    num_clientes = len(pontos)
    d_i = np.array(d_i)

    # Adiciona o ponto de partida (depósito) como o primeiro ponto
    deposito = pontos[0]
    clientes = pontos[1:].copy()  # Clientes são todos os pontos exceto o depósito
    
    for i in range(num_veiculos):
        rota = [deposito]  # Começa no depósito
        distancia_total = 0
        
        while clientes:
            ultimo_ponto = pontos[rota[-1]]
            # Encontra o cliente mais próximo
            
            proximo_cliente = min(clientes, key=lambda c: distancia[ultimo_ponto, c])
            #print(proximo_cliente)
            if distancia[ultimo_ponto, proximo_cliente] == np.inf:
                break
            elif d_i[rota+[proximo_cliente]].sum() > 7650:
                distancia[ultimo_ponto, proximo_cliente] = np.inf
            else:
                distancia_total += distancia[ultimo_ponto, proximo_cliente]
                clientes.remove(proximo_cliente)
                # Adiciona o índice do cliente na rota
                rota.append(pontos.index(proximo_cliente))

        rota.append(deposito)  # Retorna ao depósito
        rotas[i] = rota

    return rotas

In [7]:
def substituir_inf_nan(instancia):
    for key in ['d_ij', 'e_fixo_ij', 'e_ij', 'd_i']:
        if key in instancia:
            instancia[key] = np.nan_to_num(instancia[key], nan=1000, posinf=1000, neginf=1000)
    return instancia

INSTANCIAS = [substituir_inf_nan(instancia) for instancia in INSTANCIAS]

In [8]:
from copy import copy
rotas_iniciais = []
for idx, instancia in enumerate(INSTANCIAS):
    Pontos = instancia["PONTOS"]
    d_i = instancia["d_i"]
    d_ij = instancia["d_ij"]

    N = list(range(len(Pontos))) # 1 CD e 10 CLientes

    Q = 7920 # CAPACIDADE LIQUIDA DO VEICULO (KG)

    rotas = vrp_vizinho_mais_proximo(N, 20, copy(d_ij), d_i)
    #print(rotas)

    # VALIDAR ROTAS:
    
    d_i = np.array(d_i)
    rotas_dict = {"x_ijk": dict(), "f_ijk": dict(), "g_ijk": dict()}
    k = 0
    for rota in rotas:
        if len(rota)==2: continue
        #print(d_i[rota])
        if d_i[rota].sum() > Q:
            print("INVALIDO CARGA")
        bateria = 105
        for i_j in range(1, len(rota)):
            i = rota[i_j-1]
            j = rota[i_j]
            if i == 0:
                bateria -= d_ij[0, j] * 0.74
            elif j == 0:
                bateria -= d_ij[i, 0] * 0.57
            else:
                bateria -= d_ij[i, j] * 1.0

            rotas_dict["x_ijk"][(i,j,k)] = 1
    
        if bateria < 20:
            print("INVALIDO BATERIA")
            print(idx)

        k+=1
    #print(rotas_dict)

    rotas_iniciais.append(rotas_dict)

In [9]:
len(rotas_iniciais)

174

In [10]:
file_path = "./Resultados_Hibrido.xlsx"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        if "resumo" in xls.sheet_names:
            existing_summary = pd.read_excel(xls, sheet_name="resumo")
        else:
            existing_summary = pd.DataFrame()
else:
    existing_summary = pd.DataFrame()


processed_combinations = [
    (file, cluster) for file, cluster in zip(existing_summary["File"], existing_summary["cluster"])
] if not existing_summary.empty else []

In [11]:
len(processed_combinations)

72

In [12]:
# Caminho do arquivo Excel
file_path = "./Resultados_Hibrido.xlsx"

In [13]:
# PROCEDIMENTO:

import logging
import os
import traceback

log_filename = "Instancias_Hibrido.log"

# Configuração básica do logger
logging.basicConfig(
    filename=log_filename,         # Nome do arquivo de log
    level=logging.INFO,             # Nível de log (INFO, DEBUG, ERROR, etc.)
    format="%(asctime)s - %(levelname)s - %(message)s",  # Formato do log
)

dados = []
resumo = []

for idx, inst in enumerate(INSTANCIAS, start=0):

    inst_key = (inst["file"], inst["cluster"])
    print("Instância atual:", inst_key)

    # Verificar se a combinação file + cluster já foi processada
    if inst_key in processed_combinations:
        print(f"Instância {inst['file']} - Cluster {inst['cluster']} já processada. Pulando...")
        print(idx)
        continue

    # MODELO MULTIPLOS PARAMETROS MACROSCOPICOS
    # TAMANHO DE FROTA OTIMO:
    C = range(1, len(inst["PONTOS"]))
    N = range(0, len(inst["PONTOS"]))
    A = [(i,j) for i in N for j in N if i!=j]
    K = range(9)
    Q = 7920
    d_i = inst["d_i"]
    d_ij = inst["d_ij"]
    e_ij = inst['e_ij']
    e_FIXO_ij = inst['e_fixo_ij']
    SOH = 0.95
    Bmax = 105
    Bmin = 20
    Error = 2.42

    rotainicial = rotas_iniciais[idx]
    #print(rotainicial["x_ijk"])

    S = CONJUNTO_CORTES[len(C)]

    t_total = time.time()

    try:
        # ETAPA 1
        _m, x_ijk, f_ijk, g_ijk, gap, FleetSize = ModelFleetSize_Fisico(C, N, A, K, Q, d_i, SOH, Bmax, Bmin, Error, e_FIXO_ij, e_ij, rotainicial, S)
        FleetSize = int(FleetSize)
        #print(FleetSize)

        # ETAPA 2
        tempo_opt = time.time()

        _rotainicial = {"x_ijk": dict(), "f_ijk": dict(), "g_ijk": dict()}
        K_disp = []
        for i, j in A:
            for k in K:
                if x_ijk[i,j,k].X > 0.1:
                    _rotainicial["x_ijk"][(i,j,k)] = 1
                    if k not in K_disp: K_disp.append(k)

        rotainicial = {"x_ijk": dict(), "f_ijk": dict(), "g_ijk": dict()}

        for i,j,k in _rotainicial["x_ijk"]:
            if _rotainicial["x_ijk"][(i,j,k)] == 1:
                rotainicial["x_ijk"][(i,j,K_disp.index(k))] = 1

        del _rotainicial

        K = range(FleetSize)
        _m, x_ijk, f_ijk, g_ijk, gap = ModelConsumo_Fisico(C, N, A, K, Q, d_i, SOH, Bmax, Bmin, Error, FleetSize, e_FIXO_ij, e_ij, rotainicial, S)


        # PROCESSAR DADOS
        resp = {k: [] for k in K}
        dados.append([])
        
        dist = 0
                        
        consumo = _m.objVal

        lotacao_k = []
        tam_frota = 0
        for k in K:
            for j in C:
                if x_ijk[0,j,k].X > 0.1:
                    tam_frota += 1
                    lotacao_k.append(f_ijk[0,j,k].X)


        consumos_k = []
        distancias_k = []

        for k in K:  
            somatorio_consumo = 0
            somatorio_distancia = 0
            for i,j in A:  
                x_ijk_value = x_ijk[i, j, k].X  
                f_ijk_value = f_ijk[i, j, k].X 

                e_FIXO_ij_value = e_FIXO_ij[i, j]
                e_ij_value = e_ij[i, j]  
                d_ij_value = d_ij[i,j]
                
                somatorio_consumo += (x_ijk_value * e_FIXO_ij_value + e_ij_value * f_ijk_value)*1.2215 # COEFICIENTE DE 22% DE AJUSTE (ROTA PRATICADA > ROTA PLANEJADA)
                somatorio_distancia += (x_ijk_value * d_ij_value) # DISTÂNCIA JÁ FOI AJUSTADA ANTERIORMENTE

            consumos_k.append(somatorio_consumo)
            distancias_k.append(somatorio_distancia)

        for k in K:
            for i,j in A:
                if x_ijk[i,j,k].X > 0.1: 
                    resp[k].append((i,j))
                    dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "value": x_ijk[i,j,k].X, "obs": ""})
                    dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "value": f_ijk[i,j,k].X, "obs": ""})
                    dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "value": g_ijk[i,j,k].X, "obs": ""})
                    dist += d_ij[i,j]*x_ijk[i,j,k].X
                else:
                    if f_ijk[i,j,k].X > 0.1 or g_ijk[i,j,k].X>0.1:
                        dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "value": x_ijk[i,j,k].X, "obs": "Sim"})
                        dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "value": f_ijk[i,j,k].X, "obs": "Sim"})
                        dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "value": g_ijk[i,j,k].X, "obs": "Sim"})
        
        resumo.append({"File": inst["file"], "cluster": inst["cluster"], "lotacao_k": lotacao_k, "dist": dist, "dist_k": [float(d) for d in distancias_k],"consumo": consumo*1.2215, "consumo_k": [float(c) for c in consumos_k], "FleetSize": tam_frota, "t_total": time.time() - t_total, "tempo Opt": time.time() - tempo_opt, "tempo fleet": tempo_opt - t_total, "gap": gap})
        t = time.time()
        for k in K:
            if not len(resp[k]): continue
            print(resp[k])

        del _m
        del x_ijk
        del f_ijk
        del g_ijk
        del gap

        tam_frota = 0

        if os.path.exists(file_path):
            wb = load_workbook(file_path)
            if "resumo" in wb.sheetnames:
                existing_summary = pd.read_excel(file_path, sheet_name="resumo")
            else:
                existing_summary = pd.DataFrame()
        else:
            existing_summary = pd.DataFrame()

        new_summary_df = pd.DataFrame([resumo[-1]])

        if not existing_summary.empty:
            is_duplicate = (
                (existing_summary["File"] == resumo[-1]["File"]) &
                (existing_summary["cluster"] == resumo[-1]["cluster"])
            ).any()
        else:
            is_duplicate = False

        if not is_duplicate:
            existing_summary = pd.concat([existing_summary, new_summary_df], ignore_index=True)

        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            existing_summary.to_excel(writer, sheet_name="resumo", index=False)

            df = pd.DataFrame(dados[-1])

            sheet_name = f'Instancia_{idx + 1}'

            df.to_excel(writer, sheet_name=sheet_name, index=False)    
                
    except Exception as e:
        print(F"-----------------ERRO NA INSTÂNCIA {inst['file']}-----------------")
        print(f"Detalhes do erro: {str(e)}")  # Mostra o erro no console
        logging.error(f"ERRO NA INSTÂNCIA {inst['file']} - {str(e)}")
        logging.error(traceback.format_exc())  # Adiciona o traceback completo ao log
        pass

Instância atual: ('Instancias Clusters\\tabelas_2023-09-06.xlsx', 5)
Instância Instancias Clusters\tabelas_2023-09-06.xlsx - Cluster 5 já processada. Pulando...
0
Instância atual: ('Instancias Clusters\\tabelas_2023-09-06.xlsx', 1)
Instância Instancias Clusters\tabelas_2023-09-06.xlsx - Cluster 1 já processada. Pulando...
1
Instância atual: ('Instancias Clusters\\tabelas_2023-09-06.xlsx', 2)
Instância Instancias Clusters\tabelas_2023-09-06.xlsx - Cluster 2 já processada. Pulando...
2
Instância atual: ('Instancias Clusters\\tabelas_2023-09-06.xlsx', 3)
Instância Instancias Clusters\tabelas_2023-09-06.xlsx - Cluster 3 já processada. Pulando...
3
Instância atual: ('Instancias Clusters\\tabelas_2023-09-06.xlsx', 4)
Instância Instancias Clusters\tabelas_2023-09-06.xlsx - Cluster 4 já processada. Pulando...
4
Instância atual: ('Instancias Clusters\\tabelas_2023-09-15.xlsx', 4)
Instância Instancias Clusters\tabelas_2023-09-15.xlsx - Cluster 4 já processada. Pulando...
5
Instância atual: ('Ins